In [22]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
from logger import setup_logger, log_info, log_debug, log_error

In [23]:
load_dotenv()
setup_logger()

In [24]:
key = os.getenv("OPENAI_API_KEY")

In [25]:
my_client = OpenAI(api_key=key)

In [26]:
def time_this_function(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"Function `{func.__name__}` took {end - start:.3f} seconds to run.")
        return res

    return wrapper

In [27]:
@time_this_function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = my_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    log_debug(response)

    return response.choices[0].message.content


@time_this_function
def stream_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = my_client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        temperature=0,
    )

    log_debug(response)

    for resp_chunk in response:
        curr_chunk = resp_chunk.choices[0].delta.content
        if curr_chunk is not None:
            yield curr_chunk


@time_this_function
def converse(
    prompt,
    messages=None,
    model="gpt-3.5-turbo",
    max_tokens=3000,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    if messages is None:
        messages = list()

    messages.append({"role": "user", "content": prompt})

    response = (
        my_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
        )
        .choices[0]
        .message.content
    )

    messages.append({"role": "assistant", "content": response})

    return response, messages

In [28]:
@time_this_function
def create_assistant(assistant_name: str, instruction: str, model: str = "gpt-3.5-turbo"):
    my_assistant = my_client.beta.assistants.create(model=model, instructions=instruction, name=assistant_name)
    log_info(my_assistant)

    return my_assistant.id


@time_this_function
def get_assistant_from_id(assistant_id: str):
    my_assistant = my_client.beta.assistants.retrieve(assistant_id)
    log_info(my_assistant)

    return my_assistant.id


@time_this_function
def delete_assistant_with_id(assistant_id: str):
    response = my_client.beta.assistants.delete(assistant_id)
    log_info(response)

    return response.deleted


@time_this_function
def create_assistant_thread(user_msg: str):
    thread = my_client.beta.threads.create(messages=[{"role": "user", "content": user_msg}])
    log_info(thread)

    return thread.id


@time_this_function
def create_run_from_thread(thread_id: str, assistant_id: str):
    run = my_client.beta.threads.runs.create(thread_id=thread_id, assistant_id=assistant_id)
    log_info(run)

    return run.id


@time_this_function
def poll_run_till_complete(thread_id: str, run_id: str):
    while True:
        run = my_client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

        if run.status not in ["queued", "in_progress"]:
            log_info(run)
            return run.id

        time.sleep(1)


@time_this_function
def list_messages_in_thread(thread_id: str):
    messages = my_client.beta.threads.messages.list(thread_id=thread_id)
    log_info(messages)

    return messages

In [29]:
ONE_GAME_EVENTS = "It is a football match in the 2014 season of the first division league of spain. The match is played in the home turf of Real Madrid on 2014-03-23, between home team Real Madrid and away team Barcelona. On minute 4, the game is being played on Right side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was neymar who used his right foot for the shot with the secondary player involved being lionel messi.  The shot was on target and was placed at the Bottom right corner of the goal. On minute 5, the game is being played on Right side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his left foot for the shot with the secondary player involved being gareth bale.  The shot was off target and was flying Misses to the right. On minute 7, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. This resulted in a goal for Barcelona, which was accompanied by an assist via Pass. The primary player involved in the event was andres iniesta who used his left foot for the shot with the secondary player involved being lionel messi.  The shot was on target and was placed at the Top right corner of the goal. On minute 12, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his right foot for the shot with the secondary player involved being gareth bale.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 12, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his left foot for the shot with the secondary player involved being angel di maria.  The shot was off target and was flying Too high. On minute 15, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was lionel messi who used his left foot for the shot with the secondary player involved being cesc fabregas.  The shot was off target and was flying Misses to the right. On minute 19, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was neymar who used his left foot for the shot with the secondary player involved being lionel messi.  The shot was blocked by the opponent team. On minute 20, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. This resulted in a goal for Real Madrid, which was accompanied by an assist via Cross. The primary player involved in the event was karim benzema who used his head for the shot with the secondary player involved being angel di maria.  The shot was on target and was placed at the Bottom right corner of the goal. On minute 24, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. This resulted in a goal for Real Madrid, which was accompanied by an assist via Cross. The primary player involved in the event was karim benzema who used his right foot for the shot with the secondary player involved being angel di maria.  The shot was on target and was placed at the Bottom right corner of the goal. On minute 26, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his left foot for the shot with the secondary player involved being angel di maria.  The shot was blocked by the opponent team. On minute 27, the game is being played on Outside the box in the field, and Attempt happens. The primary player involved in the event was luka modric who used his left foot for the shot. The shot was off target and was flying Too high. On minute 42, the game is being played on Centre of the box in the field, and Attempt happens. This resulted in a goal for Barcelona, which was accompanied by an assist via None. The primary player involved in the event was lionel messi who used his left foot for the shot. The shot was on target and was placed at the Bottom right corner of the goal. On minute 45, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his head for the shot with the secondary player involved being daniel carvajal.  The shot was off target and was flying Misses to the right. On minute 52, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was karim benzema who used his right foot for the shot with the secondary player involved being gareth bale.  The shot was on target and was placed at the Bottom left corner of the goal. On minute 55, the game is being played on Penalty spot in the field, and Attempt happens. This resulted in a goal for Real Madrid, which was accompanied by an assist via None. The primary player involved in the event was cristiano ronaldo who used his right foot for the shot. The shot was on target and was placed at the Bottom left corner of the goal. On minute 58, the game is being played on Difficult angle on the right in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was gareth bale who used his right foot for the shot with the secondary player involved being karim benzema.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 61, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was neymar who used his left foot for the shot with the secondary player involved being cesc fabregas.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 63, the game is being played on nan in the field, and Red card happens accompanied with Sending off. The primary player involved in the event was sergio ramos. On minute 65, the game is being played on Penalty spot in the field, and Attempt happens. This resulted in a goal for Barcelona, which was accompanied by an assist via None. The primary player involved in the event was lionel messi who used his left foot for the shot. The shot was on target and was placed at the Bottom right corner of the goal. On minute 68, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was andres iniesta who used his right foot for the shot with the secondary player involved being sergio busquets.  The shot was blocked by the opponent team. On minute 74, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was dani alves who used his right foot for the shot with the secondary player involved being lionel messi.  The shot had hit the bar. On minute 82, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was javier mascherano who used his right foot for the shot with the secondary player involved being sergio busquets.  The shot was off target and was flying Bit too high. On minute 84, the game is being played on Penalty spot in the field, and Attempt happens. This resulted in a goal for Barcelona, which was accompanied by an assist via None. The primary player involved in the event was lionel messi who used his left foot for the shot. The shot was on target and was placed at the Top right corner of the goal. On minute 90, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was alexis sanchez who used his right foot for the shot with the secondary player involved being lionel messi.  The shot was off target and was flying Misses to the left. The final result of the game is 4 goals for Barcelona vs 3 goals for Real Madrid. The winner is the away team Barcelona."

In [30]:
ASSISTANT_INSTRUCTION = (
    "You are a football commentator bot in the 5 major leagues in Europe, who summarizes the major events "
    "of a game. You are given the contexts for the game, as well as the final results of the game. Use the context and search the "
    "latest data available to set up the environment, then summarize the game, and finally explain the result. While "
    "summarizing the game, you should mention the major events that had impacted the game. If you find the stadium names, "
    "or the shirt number of the players, you should also include those in your summary. Try to keep the commentary "
    "interesting and engaging for the audience while maintaining a 5 minute spoken length for the commentary. "
)

In [31]:
@time_this_function
def create_run_delete_assistant():
    assistant_name = "Soccer Commentator Bot"
    assistant_id = create_assistant(assistant_name, ASSISTANT_INSTRUCTION)
    retrieved_assistant_id = get_assistant_from_id(assistant_id)
    assert retrieved_assistant_id == assistant_id
    thread_id = create_assistant_thread(ONE_GAME_EVENTS)
    run_id = create_run_from_thread(thread_id, assistant_id)
    retrieved_run_id = poll_run_till_complete(thread_id, run_id)
    assert retrieved_run_id == run_id
    if delete_assistant_with_id(assistant_id):
        log_info(f"Deleted assistant with id: {assistant_id}")

    all_msgs = list()
    msgs = list_messages_in_thread(thread_id)
    for msg in msgs.data:
        all_msgs.append(msg.content[0].text.value)
    log_debug(all_msgs)

    return all_msgs[0]

In [32]:
@time_this_function
def assistant_test_run():
    assistant_name = "Soccer Commentator Bot"
    assistant_instruction = ASSISTANT_INSTRUCTION
    assistant_id = create_assistant(assistant_name, assistant_instruction)
    # assistant_id = "asst_8rrQUDUGROgPfMrSZxo3Fb8G"
    retrieved_assistant_id = get_assistant_from_id(assistant_id)
    assert retrieved_assistant_id == assistant_id
    # print(f"{ONE_GAME_EVENTS=}")
    response = create_run_delete_assistant(assistant_id, ONE_GAME_EVENTS)
    print(response)
    # delete_assistant_with_id(assistant_id)

In [33]:
@time_this_function
def chat_test_run():
    prompt = f"How are you feeling today?"
    response = get_completion(prompt)
    print(response)

In [34]:
# chat_test_run()

In [35]:
response = create_run_delete_assistant()

Function `create_assistant` took 0.473 seconds to run.
Function `get_assistant_from_id` took 0.196 seconds to run.
Function `create_assistant_thread` took 0.312 seconds to run.
Function `create_run_from_thread` took 0.429 seconds to run.
Function `poll_run_till_complete` took 7.288 seconds to run.
Function `delete_assistant_with_id` took 0.209 seconds to run.
Function `list_messages_in_thread` took 0.213 seconds to run.
Function `create_run_delete_assistant` took 9.125 seconds to run.


In [37]:
print(response)

Welcome to today's thrilling match between two Spanish giants, Real Madrid and Barcelona, at the Santiago Bernabeu Stadium. The atmosphere is electric as fans are on the edge of their seats for this highly anticipated clash.

The game started with a bang as Neymar opens the scoring for Barcelona in the 7th minute with a precise shot into the top right corner, assisted by Iniesta. Real Madrid quickly responds with Benzema equalizing in the 20th minute, heading in a cross from Di Maria. Benzema becomes a key player, scoring again in the 24th minute, this time with a right-footed shot at the bottom right corner from another Di Maria cross.

The first half continues to be intense as Messi scores for Barcelona in the 42nd minute, putting them level with Real Madrid. The teams head into halftime with a 2-2 scoreline.

The second half sees Cristiano Ronaldo converting a penalty in the 55th minute to put Real Madrid in the lead. However, the game takes a turn when Sergio Ramos receives a red c